# S2W5D2: 灵魂核心 - Scaled Dot-Product Attention

## 1 核心概念：Q\K\V到底是什么？

Tranformer的灵感来源于检索系统（比如你查字典，或者用搜索引擎）。想象一下去图书馆找书：

1. **Query**($Q$)：你手里的“索书号”或者你想搜的关键词（比如“人工智能”）。
2. **Key**($K$)：图书馆每本书脊上的“标签”或“分类号”。
3. **Value**($V$)：书里面的具体**内容**。

**注意力机制的过程**：

1. 你拿着你的$Q$，去和所有的$K$进行**比对**（计算相似度）。
2. 如果 $Q$ 和某个$K$很像（相似度高），你就给这本书分配更多的**关注度**（权重）。
3. 最后，你读到的知识，是所有书的内容 $V$ 根据关注度加权融合后的结果。

> **在Self-Attention中**，$Q, K, V$其实都是同一个东西（输入$X$）经过三种不同的线性层变换来的，也就是**我自己查询我自己，看看我和上下文中哪些词关系最紧密**。

## 2 数学公式

必须牢记这个公式：

$$\text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V$$

解析：

1. **$QK^T$**: 矩阵乘法。计算 **Query** 和 **Key** 的相似度。
    - $Q$ 维度: $[Batch, Seq\_Len, D_k]$
    - $K^T$ (转置后) 维度: $[Batch, D_k, Seq\_Len]$
    
    - 结果维度: $[Batch, Seq\_Len, Seq\_Len]$ —— **这是一个 $N \times N$ 的分数矩阵 (Attention Scores)**。
2. **$\frac{1}{\sqrt{d_k}}$ (Scale)**: **缩放**。为什么要除以根号 $d_k$？（面试必问，稍后详解）。
3. **Softmax**: **归一化**。把分数变成概率（让每一行的和为 1）。
4. **$\dots V$**: **加权求和**。用算出来的概率去乘以 $V$，得到最终输出。

## 3 手写代码实现(Hand-Written Implementation)

这是一段 **"必须背诵"** 级别的代码。我们将实现一个独立的函数 `scaled_dot_product_attention`。

In [1]:
import math
import torch
import torch.nn.functional as F

def scaled_dot_product_attention(query, key, value, mask=None):
    """
    计算Scaled Dot-Product Attention
    
    Args:
        query:  [batch_size, n_heads, len_q, d_k](为了通用，我们在这里假设又多头参数，单头也是一样的逻辑)
        key:    [batch_size, n_heads, len_k, d_k]
        value:  [batch_size, n_heads, len_v, d_v]

    Returns:
        output: [batch_size, n_heads, len_q, d_v]
        attn_weights: [batch_size, n_heads, len_q, len_k]
    """

    # 1. 获取d_k（用于缩放）
    d_k = query.size(-1)

    # 2. 计算 QK^T (Matmul)
    # query: [..., len_q, d_k]
    # key.transpose(-2, -1): [..., d_k, len_k] 将最后两个维度交换，做转置
    # scores: [..., len_q, len_k]
    scores = torch.matmul(query, key.transpose(-2, -1))

    #3. 缩放（Scale）
    # 这里的数学意义是防止点积结果过大导致 Softmax 梯度消失
    scores = scores / math.sqrt(d_k)
    
    # 4. Mask (可选)
    # 如果有 mask，把 mask 为 0 的位置的分数设为负无穷大 (-1e9)
    # 这样 Softmax 之后，这些位置的概率就会变成 0
    if mask is not None:
        # mask == 0的地方，填上-1e9
        scores = scores.masked_fill(mask == 0, -1e9)

    # 5. Softmax 归一化
    # dim=-1 表示对每一行的最后一个维度(即针对所有 key)做归一化
    attn_weights = F.softmax(scores, dim=-1)

    # 6. 加权求和 (Weighted Sum)
    # attn_weights: [..., len_q, len_k]
    # value:        [..., len_k, d_v]
    # output:       [..., len_q, d_v]
    output = torch.matmul(attn_weights, value)
    
    return output, attn_weights

## 4 单元测试

测试代码

In [7]:
batch_size = 2
n_heads = 1 # 暂时当做单头来看
seq_len = 3
d_k = 4 # 假设特征维度是 4

# 随机生成Q, K, V
# 实际中Q, K, V通常来自同一输入X经过Linear变换，这里简化直接随机生成
q = torch.randn(batch_size, n_heads, seq_len, d_k)
k = torch.randn(batch_size, n_heads, seq_len, d_k)
v = torch.randn(batch_size, n_heads, seq_len, d_k)

print(f"Query shape: {q.shape}")
print(f"Key shape:   {k.shape}")

# 调用函数
output, weights = scaled_dot_product_attention(q, k, v)

print("----------------")
print(f"Output shape:  {output.shape} (应该和 Q 保持一致)")
print(f"Weights shape: {weights.shape} (应该是 [Batch, Heads, Seq, Seq])")

# 验证 Softmax 每一行的和是否为 1
print("----------------")
print("Sum of weights (Check Softmax):")
print(weights[0][0].sum(dim=-1)) # 应该全是 1.0000

Query shape: torch.Size([2, 1, 3, 4])
Key shape:   torch.Size([2, 1, 3, 4])
----------------
Output shape:  torch.Size([2, 1, 3, 4]) (应该和 Q 保持一致)
Weights shape: torch.Size([2, 1, 3, 3]) (应该是 [Batch, Heads, Seq, Seq])
----------------
Sum of weights (Check Softmax):
tensor([1.0000, 1.0000, 1.0000])


## 5 深度解析 & 面试八股 (Interview Deep Dive)

做完实验后，请务必阅读以下三个问题，这是面试官最爱问的“为什么”。

### Q1: 为什么要除以 $\sqrt{d_k}$？(Scaling 的作用)

  * **现象**: 如果 $d_k$ (向量维度) 很大，点积 $Q \cdot K$ 的结果方差会变大，数值可能会非常大（比如几百）。
  * **后果**: 把很大的数喂给 Softmax 函数（比如 `softmax([100, 200])`），梯度会变得极小（趋近于 0），导致**梯度消失**，模型无法训练。
  * **解决**: 除以 $\sqrt{d_k}$ 可以将数值拉回到均值为 0、方差为 1 的范围内，保证梯度平稳流转。

### Q2: 这里的 Mask 是干什么用的？

  * **Padding Mask**: 输入句子长度不一（比如 "Hello world \<pad\> \<pad\>"）。所有的 `<pad>` 都是没有意义的占位符。我们不希望模型把注意力浪费在 `<pad>` 上。所以我们在计算 Softmax 之前，把 `<pad>` 对应的分数设为 `-1e9`，这样 `softmax(-1e9) ≈ 0`，完美忽略。

### Q3: 为什么 $QK^T$ 的结果是一个 $N \times N$ 的矩阵？

  * 因为这是一个 **"All-to-All"** 的比较。
  * 矩阵的第 $i$ 行，第 $j$ 列的数值，代表了 **"第 $i$ 个词 对 第 $j$ 个词 的关注度"**。
  * 这就是 Self-Attention 能够捕捉全局依赖的原因：无论两个词距离多远，它们都在这个矩阵里直接发生了交互。

## 🚀 今日任务

1.  **手敲代码**: 不要复制，请手动输入一遍 `scaled_dot_product_attention` 函数，体会 `transpose` 和 `matmul` 的过程。
2.  **运行测试**: 确保 Output Shape 正确，且 Softmax 求和为 1。
3.  **思考**: 如果 `query` 和 `key` 是完全一样的矩阵，`weights` 对角线上的数值会比较大吗？（提示：自己和自己通常最相似）。

等你跑通代码，我们就完成了 Transformer 的心脏手术！明天我们将把这个心脏复制 8 份，组装成 **Multi-Head Attention**。

### 1. 【热身】LeetCode 867. 转置矩阵 (Transpose Matrix)
* **难度**: 🟢 简单
* **链接**: [https://leetcode.cn/problems/transpose-matrix/](https://leetcode.cn/problems/transpose-matrix/)
* **🔥 为什么选这道题**:
    * **直接对应代码**: 这道题就是今天 Attention 代码中 `key.transpose(-2, -1)` 的物理实现！
    * **核心任务**: 把 $M \times N$ 的矩阵变成 $N \times M$。
    * **注意事项**: 今天的课程里通过 PyTorch 一行代码解决了，但这道题要求你理解底层的索引变换规则 `new_matrix[j][i] = matrix[i][j]`。

### 2. 【进阶】LeetCode 73. 矩阵置零 (Set Matrix Zeroes)
* **难度**: 🟡 中等
* **链接**: [https://leetcode.cn/problems/set-matrix-zeroes/](https://leetcode.cn/problems/set-matrix-zeroes/)
* **🔥 为什么选这道题**:
    * **关联概念**: **Mask (掩码)**。
    * **逻辑映射**: 在 Attention 中，我们用 `masked_fill` 把不需要关注的地方（Padding）变成负无穷大（相当于“置零”或无效化）。
    * **挑战**: 这道题要求你把矩阵中所有 `0` 所在的**整行**和**整列**都设为 `0`。这很像 Attention 中的广播机制（Broadcasting）—— 一个点影响了一整行/列。
    * **要求**: 尝试用 $O(1)$ 的空间原地解决（利用第一行/第一列当做标记位），这能极大地锻炼你的**原地下标操作能力**。

### 3. 【挑战】LeetCode 54. 螺旋矩阵 (Spiral Matrix)
* **难度**: 🟡 中等
* **链接**: [https://leetcode.cn/problems/spiral-matrix/](https://leetcode.cn/problems/spiral-matrix/)
* **🔥 为什么选这道题**:
    * **核心能力**: **极致的索引控制 (Index Manipulation)**。
    * **深度学习映射**: 虽然 Transformer 不会螺旋读取数据，但这道题考察的是你处理边界条件（Boundary Conditions）和复杂遍历逻辑的能力。
    * **痛点**: 写这道题很容易把自己绕晕（什么时候拐弯？边界在哪里？）。如果你能一次 AC 这道题，说明你对张量的切片（Slicing）和索引逻辑已经有了非常清晰的头脑，Debug 复杂的 Model 代码就不在话下了。